In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio

Preprocessing

- Resolve the image paths
- Extract the labels from the path
- Move the data -> pandas dataframe
- Generate batcches of tensor image data with real-time data augmentation using ImageDataGenerator

Data Retrieval

In [3]:
dataset_path = Path("")